# EDA003を整理（EDA003をutils.pyにしたものを検証）


In [1]:
import os
import pandas as pd
import numpy as np
from datetime import datetime
import folium

In [2]:
class CFG:
    make_folder = True
    note_num = "StackingOpt/EDA004"
    seed = 42


OUTPUT_DIR = f'H:/study/output/{CFG.note_num}/'

if CFG.make_folder:
    if not os.path.exists(OUTPUT_DIR):
        os.makedirs(OUTPUT_DIR)


In [3]:
unique_id = ['10000095', '10000269', '1020000002', '1110000001', '1110000010', '1110000011', '1110000012', '1110000013', '1110000014', '1110000015', '1160000025', '1160000090', '1160000091', '1160000182', '1160000185', '1160000253', '1160000387', '1160000402', '1160000419', '1160000420', '1160000423', '1270000026', '1280000048', '1550000001', '1650000004', '1680000001', '1680000002', '1680000003', '1680000004', '1680000010', '1680000017', '1680000021', '1680000033', '1680000047', '1680000054', '1680000057', '1680000063', '1680000067', '1680000080', '1680000081', '1680000097', '1680000107', '1680000108', '1680000112', '1680000151', '1680000152', '1680000213', '1680000216', '1680000217', '1680000218', '1680000223', '1680000228', '1680000285', '1680000287', '1680000327', '1680000364', '2220000001', '2220000002', '2220000003', '2730000001', '2910000002', '3000000007', '3000000012', '3000000042', '5000000044', '5000000045', '6000000016', '6000000017', '6060000016', '6060000017', '6060000018', '6170000016', '6170000123', '6170000124', '6170000125', '6620000065', '6620000066', '6620000088', '6620000089', '6620000111', '6620000117', '6620000118', '6620000121', '6620000122', '6620000123', '6620000124', '6620000131', '6620000132', '6910000180', '6910000198', '6910000200', '6910000206', '6910000216', '6910000217', '6910000239', '6910000240', '6910000249', '6910000250', '6910000421', '6910000424', '6910000425', '6910000469', '6910000470']
unique_id = [int(i) for i in unique_id]
to_unique_id = [str(num).zfill(10) for num in unique_id]

# preprocessing_data\1_twoweeks_nv より

In [4]:
import utils

df = utils.get_preprocessing_data(to_unique_id)

# prediction_data\parameter_lambda_0.019_iterate_1000_p3\env

In [5]:
lat = pd.read_csv(r"H:\study\preprocessing_data\3_mesh_place\lati_zenkoku.csv", header=None, index_col=None)
lon = pd.read_csv(r"H:\study\preprocessing_data\3_mesh_place\long_zenkoku.csv", header=None, index_col=None)
id_all_data = pd.read_csv(r"H:\study\preprocessing_data\id_all_data.csv", encoding="shift_jis")
id_data = id_all_data[id_all_data.id.isin(unique_id)].reset_index(drop=True)


In [6]:
import datetime

start_date = datetime.datetime(2013, 8, 15, 6, 30)
end_date = datetime.datetime(2014, 7, 31, 18, 0)
# start_date = datetime.datetime(2014, 7, 1, 6, 30)
# end_date = datetime.datetime(2014, 7, 31, 18, 0)

delta = datetime.timedelta(minutes=30)

date_range = []
date = start_date
while date <= end_date:
    if date.hour > 6 and date.hour < 18:
        date_range.append(date)
    elif date.hour == 18 and date.minute == 0:
        date_range.append(date)
    elif date.hour == 6 and date.minute == 30:
        date_range.append(date)
    date += delta


In [7]:
mesh_data_list,pred_data_list = [],[]
for i,dt in enumerate(date_range):
    mesh_data,pred_data = utils.get_prediction_data(id_data,lat,lon,dt)
    mesh_data_list.append(mesh_data)
    pred_data_list.append(pred_data)

    mesh_data.to_csv(OUTPUT_DIR+f"mesh_data_{dt.strftime('%Y%m%d%H%M')}.csv")
    if i%int(len(date_range)*0.1)==0:
        print(f"{i/len(date_range)*100}%_finish")

preds = pd.concat(pred_data_list).reset_index(drop=True)
preds.to_csv(OUTPUT_DIR+"preds.csv",index=False)

0.0%_finish
9.995251661918328%_finish
19.990503323836656%_finish
29.985754985754987%_finish
39.98100664767331%_finish
49.97625830959164%_finish
59.971509971509974%_finish
69.96676163342829%_finish
79.96201329534662%_finish
89.95726495726495%_finish
99.95251661918329%_finish


# 予測精度を比較

In [44]:
preds = pd.read_csv(OUTPUT_DIR+"preds.csv")
preds["datetime"] = pd.to_datetime(preds["datetime"])

In [45]:
import utils

df = utils.get_preprocessing_data(to_unique_id)
df["datetime"] = pd.to_datetime(df["datetime"])

df = df.merge(id_data[["id","id_lat_mesh","id_lng_mesh","pvrate","observed_max"]],on=["id"],how="left")
df = df[df.datetime.isin(date_range)]
df = df.merge(preds,on=["datetime","id_lat_mesh","id_lng_mesh"],how="left")

df["pred*two_weeks_max"] = df["pred"]*df["two_weeks_max"]
df["nv*twoweeks_max"] = df["nv"]*df["two_weeks_max"]

df["APE"] = np.abs(df["pred*two_weeks_max"]-df["nv*twoweeks_max"])/df["observed_max"]*100

In [46]:
df["APE"].mean()

4.622661015944398

In [47]:
df["month"] = df.datetime.dt.month
for month,group in df.groupby("month"):
    print(f"{month}月",group["APE"].mean() )
    print()

1月 3.004600552943324

2月 4.531547490686455

3月 3.4208157122279808

4月 4.661107656975637

5月 5.445719045449569

6月 6.36660266111151

7月 6.4246202475227125

8月 5.074576764676934

9月 5.984355188662475

10月 4.745663452336866

11月 3.1692568426963574

12月 2.7771447530777835



# nv,meshの確認
- nv_meshは同緯度経度、時間のmeshの平均
- nv_meshを理想的に予測できたとして、平均APE 1.08%

In [48]:
mesh_place = pd.read_csv(OUTPUT_DIR + "mesh_place.csv")
mesh_place

,id_lat_mesh,id_lng_mesh,nv_mesh,datetime
0,31.50,131.06,1.000,2013-08-15 05:00:00
1,31.52,130.52,0.648,2013-08-15 05:00:00
2,31.66,130.32,0.777,2013-08-15 05:00:00
3,32.00,130.98,1.000,2013-08-15 05:00:00
4,32.02,130.74,0.982,2013-08-15 05:00:00
...,...,...,...,...
19755486,38.68,141.20,0.845,2014-07-31 18:30:00
19755487,38.70,141.10,0.871,2014-07-31 18:30:00
19755488,38.82,141.56,0.871,2014-07-31 18:30:00
19755489,38.90,141.58,0.871,2014-07-31 18:30:00


In [49]:
df["datetime"] = pd.to_datetime(df["datetime"])
mesh_place["datetime"] = pd.to_datetime(mesh_place["datetime"])
df = df.merge(mesh_place,on=["id_lat_mesh","id_lng_mesh","datetime"],how="left")

In [50]:
df[(df["id_lat_mesh"]==35.66)&(df["id_lng_mesh"]==140.10)&(df["datetime"]=="2013-08-15 06:30:00")]

,year,month,day,hour,flag,nv,two_weeks_max,id,minute,datetime,id_lat_mesh,id_lng_mesh,pvrate,observed_max,pred,pred*two_weeks_max,nv*twoweeks_max,APE,nv_mesh
33579,2013,8,15,6,2,0.910,5577.6,1110000010,30,2013-08-15 06:30:00,35.66,140.1,20.0,18208.5904,0.691807,3858.624564,5075.6160,6.683611,0.873
42051,2013,8,15,6,2,0.892,5577.6,1110000011,30,2013-08-15 06:30:00,35.66,140.1,20.0,17715.6980,0.691807,3858.624564,4975.2192,6.302854,0.873
50523,2013,8,15,6,2,0.898,4934.4,1110000012,30,2013-08-15 06:30:00,35.66,140.1,20.0,17873.1820,0.691807,3413.654089,4431.0912,5.692535,0.873
58995,2013,8,15,6,2,0.901,4974.6,1110000013,30,2013-08-15 06:30:00,35.66,140.1,20.0,17745.7280,0.691807,3441.464744,4482.1146,5.864227,0.873
67467,2013,8,15,6,2,0.890,5688.2,1110000014,30,2013-08-15 06:30:00,35.66,140.1,20.0,17971.4106,0.691807,3935.138455,5062.4980,6.273072,0.873
75939,2013,8,15,6,2,0.886,5728.4,1110000015,30,2013-08-15 06:30:00,35.66,140.1,20.0,18188.3702,0.691807,3962.949109,5075.3624,6.116069,0.873
577783,2013,8,15,6,2,0.810,3707.2,6170000123,30,2013-08-15 06:30:00,35.66,140.1,16.5,13399.7864,0.691807,2564.668134,3002.8320,3.269932,0.873
586248,2013,8,15,6,2,0.810,958.4,6170000124,30,2013-08-15 06:30:00,35.66,140.1,9.0,3801.5978,0.691807,663.028145,776.3040,2.979691,0.873
594714,2013,8,15,6,2,0.863,3747.4,6170000125,30,2013-08-15 06:30:00,35.66,140.1,16.5,13654.1322,0.691807,2592.478788,3234.0062,4.698412,0.873


In [51]:
df[(df["id_lat_mesh"]==35.66)&(df["id_lng_mesh"]==140.10)&(df["datetime"]=="2013-08-15 06:30:00")].nv.mean()

0.8733333333333333

In [52]:
import utils
import pandas as pd
import datetime


lat = pd.read_csv(r"H:\study\preprocessing_data\3_mesh_place\lati_zenkoku.csv", header=None, index_col=None)
lon = pd.read_csv(r"H:\study\preprocessing_data\3_mesh_place\long_zenkoku.csv", header=None, index_col=None)
id_all_data = pd.read_csv(r"H:\study\preprocessing_data\id_all_data.csv", encoding="shift_jis")

unique_id = ['10000095', '10000269', '1020000002', '1110000001', '1110000010', '1110000011', '1110000012', '1110000013', '1110000014', '1110000015', '1160000025', '1160000090', '1160000091', '1160000182', '1160000185', '1160000253', '1160000387', '1160000402', '1160000419', '1160000420', '1160000423', '1270000026', '1280000048', '1550000001', '1650000004', '1680000001', '1680000002', '1680000003', '1680000004', '1680000010', '1680000017', '1680000021', '1680000033', '1680000047', '1680000054', '1680000057', '1680000063', '1680000067', '1680000080', '1680000081', '1680000097', '1680000107', '1680000108', '1680000112', '1680000151', '1680000152', '1680000213', '1680000216', '1680000217', '1680000218', '1680000223', '1680000228', '1680000285', '1680000287', '1680000327', '1680000364', '2220000001', '2220000002', '2220000003', '2730000001', '2910000002', '3000000007', '3000000012', '3000000042', '5000000044', '5000000045', '6000000016', '6000000017', '6060000016', '6060000017', '6060000018', '6170000016', '6170000123', '6170000124', '6170000125', '6620000065', '6620000066', '6620000088', '6620000089', '6620000111', '6620000117', '6620000118', '6620000121', '6620000122', '6620000123', '6620000124', '6620000131', '6620000132', '6910000180', '6910000198', '6910000200', '6910000206', '6910000216', '6910000217', '6910000239', '6910000240', '6910000249', '6910000250', '6910000421', '6910000424', '6910000425', '6910000469', '6910000470']
unique_id = [int(i) for i in unique_id]
id_data = id_all_data[id_all_data.id.isin(unique_id)].reset_index(drop=True)

dt= datetime.datetime(2013, 8, 15, 6, 30)
df2 = utils.get_interpolated_mesh(id_data,lat,lon,dt)

In [64]:
lat.iloc[0][lat.iloc[0]==35.66].index.values

array([223], dtype=int64)

In [67]:
list(lat.iloc[0][lat.iloc[0]==35.66].index),list(lon.iloc[:,0][lon.iloc[:,0]==140.10].index)

([223], [525])

In [71]:
df2

,223,224,225,226,227,228,229,230,231,232,233,234
515,0.73000,0.41700,0.65542,0.72541,0.74431,0.75389,0.69025,0.74000,0.77077,0.77572,0.74674,0.71044
516,0.75632,0.66100,0.67500,0.76300,0.72000,0.85000,0.55500,0.73800,0.81122,0.83600,0.75849,0.67894
517,0.78611,0.68943,0.61780,0.66783,0.67100,0.67903,0.58200,0.77275,1.00100,0.86050,0.79500,0.66950
518,0.81253,0.74180,0.54600,0.62353,0.73900,0.69979,0.70517,0.77000,0.84300,0.81000,0.72825,0.59900
519,0.84454,0.81284,0.67110,0.62931,0.62200,0.72177,0.79200,0.83800,0.88657,0.85095,0.75209,0.65100
520,0.88007,0.89500,0.73843,0.64480,0.65400,0.72414,0.78488,0.85331,1.00100,0.87538,0.77467,0.69400
521,0.89720,1.00000,0.76436,0.57200,0.67517,0.73400,0.78712,0.82500,0.87537,0.81033,0.74011,0.71593
522,0.90186,0.77500,0.74904,0.70400,0.72399,0.77057,0.81900,0.79967,0.80196,0.76020,0.74650,0.73658
523,0.95700,0.83348,0.75739,0.69978,0.74444,0.78875,0.79958,0.78152,0.75500,0.76498,0.76366,0.75921
524,0.86030,0.81951,0.78000,0.66700,0.76994,0.80182,0.80582,0.79711,0.78821,0.78981,0.78872,0.78120


In [78]:
df["nv_mesh*two_weeks_max"] = df["nv_mesh"] * df["two_weeks_max"]

In [80]:
df["APE"] = np.abs(df["nv_mesh*two_weeks_max"]-df["nv*twoweeks_max"])/df["observed_max"]*100

In [81]:
df["APE"].mean()

1.087652057139833

# 欠損値補間のデータを抽出

In [82]:
import utils
import pandas as pd
import datetime


lat = pd.read_csv(r"H:\study\preprocessing_data\3_mesh_place\lati_zenkoku.csv", header=None, index_col=None)
lon = pd.read_csv(r"H:\study\preprocessing_data\3_mesh_place\long_zenkoku.csv", header=None, index_col=None)
id_all_data = pd.read_csv(r"H:\study\preprocessing_data\id_all_data.csv", encoding="shift_jis")

unique_id = ['10000095', '10000269', '1020000002', '1110000001', '1110000010', '1110000011', '1110000012', '1110000013', '1110000014', '1110000015', '1160000025', '1160000090', '1160000091', '1160000182', '1160000185', '1160000253', '1160000387', '1160000402', '1160000419', '1160000420', '1160000423', '1270000026', '1280000048', '1550000001', '1650000004', '1680000001', '1680000002', '1680000003', '1680000004', '1680000010', '1680000017', '1680000021', '1680000033', '1680000047', '1680000054', '1680000057', '1680000063', '1680000067', '1680000080', '1680000081', '1680000097', '1680000107', '1680000108', '1680000112', '1680000151', '1680000152', '1680000213', '1680000216', '1680000217', '1680000218', '1680000223', '1680000228', '1680000285', '1680000287', '1680000327', '1680000364', '2220000001', '2220000002', '2220000003', '2730000001', '2910000002', '3000000007', '3000000012', '3000000042', '5000000044', '5000000045', '6000000016', '6000000017', '6060000016', '6060000017', '6060000018', '6170000016', '6170000123', '6170000124', '6170000125', '6620000065', '6620000066', '6620000088', '6620000089', '6620000111', '6620000117', '6620000118', '6620000121', '6620000122', '6620000123', '6620000124', '6620000131', '6620000132', '6910000180', '6910000198', '6910000200', '6910000206', '6910000216', '6910000217', '6910000239', '6910000240', '6910000249', '6910000250', '6910000421', '6910000424', '6910000425', '6910000469', '6910000470']
unique_id = [int(i) for i in unique_id]
id_data = id_all_data[id_all_data.id.isin(unique_id)].reset_index(drop=True)

dt= datetime.datetime(2013, 8, 15, 6, 30)
df = utils.get_interpolated_mesh(id_data,lat,lon,dt)

In [7]:
import datetime

start_date = datetime.datetime(2013, 8, 15, 6, 30)
end_date = datetime.datetime(2014, 7, 31, 18, 0)
# start_date = datetime.datetime(2014, 7, 1, 6, 30)
# end_date = datetime.datetime(2014, 7, 31, 18, 0)

delta = datetime.timedelta(minutes=30)

date_range = []
date = start_date
while date <= end_date:
    if date.hour > 6 and date.hour < 18:
        date_range.append(date)
    elif date.hour == 18 and date.minute == 0:
        date_range.append(date)
    elif date.hour == 6 and date.minute == 30:
        date_range.append(date)
    date += delta


In [8]:
mesh_data_list,pred_data_list = [],[]
for i,dt in enumerate(date_range):
    mesh_data = utils.get_interpolated_mesh(id_data,lat,lon,dt)
    mesh_data_list.append(mesh_data)

    mesh_data.to_csv(OUTPUT_DIR+f"interpolated_mesh_data_{dt.strftime('%Y%m%d%H%M')}.csv")
    if i%int(len(date_range)*0.1)==0:
        print(f"{i/len(date_range)*100}%_finish")

0.0%_finish
9.995251661918328%_finish
19.990503323836656%_finish
29.985754985754987%_finish
39.98100664767331%_finish
49.97625830959164%_finish
59.971509971509974%_finish
69.96676163342829%_finish
79.96201329534662%_finish
89.95726495726495%_finish
99.95251661918329%_finish


In [11]:
mesh_data_list[0]

,223,224,225,226,227,228,229,230,231,232,233,234
515,0.73000,0.41700,0.65542,0.72541,0.74431,0.75389,0.69025,0.74000,0.77077,0.77572,0.74674,0.71044
516,0.75632,0.66100,0.67500,0.76300,0.72000,0.85000,0.55500,0.73800,0.81122,0.83600,0.75849,0.67894
517,0.78611,0.68943,0.61780,0.66783,0.67100,0.67903,0.58200,0.77275,1.00100,0.86050,0.79500,0.66950
518,0.81253,0.74180,0.54600,0.62353,0.73900,0.69979,0.70517,0.77000,0.84300,0.81000,0.72825,0.59900
519,0.84454,0.81284,0.67110,0.62931,0.62200,0.72177,0.79200,0.83800,0.88657,0.85095,0.75209,0.65100
520,0.88007,0.89500,0.73843,0.64480,0.65400,0.72414,0.78488,0.85331,1.00100,0.87538,0.77467,0.69400
521,0.89720,1.00000,0.76436,0.57200,0.67517,0.73400,0.78712,0.82500,0.87537,0.81033,0.74011,0.71593
522,0.90186,0.77500,0.74904,0.70400,0.72399,0.77057,0.81900,0.79967,0.80196,0.76020,0.74650,0.73658
523,0.95700,0.83348,0.75739,0.69978,0.74444,0.78875,0.79958,0.78152,0.75500,0.76498,0.76366,0.75921
524,0.86030,0.81951,0.78000,0.66700,0.76994,0.80182,0.80582,0.79711,0.78821,0.78981,0.78872,0.78120


In [10]:
mesh_data_list[0].shape

(15, 12)

In [86]:
lat

,0,1,2,3,4,5,6,7,8,9,...,421,422,423,424,425,426,427,428,429,430
0,31.2,31.22,31.24,31.26,31.28,31.3,31.32,31.34,31.36,31.38,...,39.62,39.64,39.66,39.68,39.7,39.72,39.74,39.76,39.78,39.8
1,31.2,31.22,31.24,31.26,31.28,31.3,31.32,31.34,31.36,31.38,...,39.62,39.64,39.66,39.68,39.7,39.72,39.74,39.76,39.78,39.8
2,31.2,31.22,31.24,31.26,31.28,31.3,31.32,31.34,31.36,31.38,...,39.62,39.64,39.66,39.68,39.7,39.72,39.74,39.76,39.78,39.8
3,31.2,31.22,31.24,31.26,31.28,31.3,31.32,31.34,31.36,31.38,...,39.62,39.64,39.66,39.68,39.7,39.72,39.74,39.76,39.78,39.8
4,31.2,31.22,31.24,31.26,31.28,31.3,31.32,31.34,31.36,31.38,...,39.62,39.64,39.66,39.68,39.7,39.72,39.74,39.76,39.78,39.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
596,31.2,31.22,31.24,31.26,31.28,31.3,31.32,31.34,31.36,31.38,...,39.62,39.64,39.66,39.68,39.7,39.72,39.74,39.76,39.78,39.8
597,31.2,31.22,31.24,31.26,31.28,31.3,31.32,31.34,31.36,31.38,...,39.62,39.64,39.66,39.68,39.7,39.72,39.74,39.76,39.78,39.8
598,31.2,31.22,31.24,31.26,31.28,31.3,31.32,31.34,31.36,31.38,...,39.62,39.64,39.66,39.68,39.7,39.72,39.74,39.76,39.78,39.8
599,31.2,31.22,31.24,31.26,31.28,31.3,31.32,31.34,31.36,31.38,...,39.62,39.64,39.66,39.68,39.7,39.72,39.74,39.76,39.78,39.8


In [34]:
lat = pd.read_csv(r"H:\study\preprocessing_data\3_mesh_place\lati_zenkoku.csv", header=None, index_col=None)
lon = pd.read_csv(r"H:\study\preprocessing_data\3_mesh_place\long_zenkoku.csv", header=None, index_col=None)
id_all_data = pd.read_csv(r"H:\study\preprocessing_data\id_all_data.csv", encoding="shift_jis")

unique_id = ['10000095', '10000269', '1020000002', '1110000001', '1110000010', '1110000011', '1110000012', '1110000013', '1110000014', '1110000015', '1160000025', '1160000090', '1160000091', '1160000182', '1160000185', '1160000253', '1160000387', '1160000402', '1160000419', '1160000420', '1160000423', '1270000026', '1280000048', '1550000001', '1650000004', '1680000001', '1680000002', '1680000003', '1680000004', '1680000010', '1680000017', '1680000021', '1680000033', '1680000047', '1680000054', '1680000057', '1680000063', '1680000067', '1680000080', '1680000081', '1680000097', '1680000107', '1680000108', '1680000112', '1680000151', '1680000152', '1680000213', '1680000216', '1680000217', '1680000218', '1680000223', '1680000228', '1680000285', '1680000287', '1680000327', '1680000364', '2220000001', '2220000002', '2220000003', '2730000001', '2910000002', '3000000007', '3000000012', '3000000042', '5000000044', '5000000045', '6000000016', '6000000017', '6060000016', '6060000017', '6060000018', '6170000016', '6170000123', '6170000124', '6170000125', '6620000065', '6620000066', '6620000088', '6620000089', '6620000111', '6620000117', '6620000118', '6620000121', '6620000122', '6620000123', '6620000124', '6620000131', '6620000132', '6910000180', '6910000198', '6910000200', '6910000206', '6910000216', '6910000217', '6910000239', '6910000240', '6910000249', '6910000250', '6910000421', '6910000424', '6910000425', '6910000469', '6910000470']
unique_id = [int(i) for i in unique_id]
id_data = id_all_data[id_all_data.id.isin(unique_id)].reset_index(drop=True)

min_lat,max_lat = id_data.id_lat_mesh.min(),id_data.id_lat_mesh.max()
min_lng,max_lng = id_data.id_lng_mesh.min(),id_data.id_lng_mesh.max()

userow = (lon.iloc[:,0]>=min_lng)&(lon.iloc[:,0]<=max_lng)
usecol = (lat.iloc[0]>=min_lat)&(lat.iloc[0]<=max_lat)
lat = lat.loc[userow,usecol]
lon = lon.loc[userow,usecol]

In [37]:
mesh_lat_lng = pd.DataFrame(zip(lat.to_numpy().reshape(-1),lon.to_numpy().reshape(-1)),columns=['id_lat_mesh','id_lng_mesh'])
mesh_lat_lng["num"] = mesh_lat_lng.index

In [38]:
id_data[['id_lat_mesh','id_lng_mesh']]

,id_lat_mesh,id_lng_mesh
0,35.74,140.00
1,35.70,139.96
2,35.72,140.12
3,35.72,140.02
4,35.66,140.10
...,...,...
98,35.74,139.96
99,35.74,139.92
100,35.74,139.94
101,35.82,139.88


In [39]:
id_data_rounded = id_data.copy()
id_data_rounded["id_lat_mesh"] = id_data_rounded["id_lat_mesh"].round(2)
id_data_rounded["id_lng_mesh"] = id_data_rounded["id_lng_mesh"].round(2)

mesh_lat_lng_unique_rounded = mesh_lat_lng.drop_duplicates().copy()
mesh_lat_lng_unique_rounded["id_lat_mesh"] = mesh_lat_lng_unique_rounded["id_lat_mesh"].round(2)
mesh_lat_lng_unique_rounded["id_lng_mesh"] = mesh_lat_lng_unique_rounded["id_lng_mesh"].round(2)

result_rounded = id_data_rounded.merge(mesh_lat_lng_unique_rounded, on=["id_lat_mesh", "id_lng_mesh"], how="left")

result_rounded 

,id,id_lat,id_lng,id_lat_mesh,id_lng_mesh,id_prefecture,pvrate,observed_max,num
0,10000095,35.74936,139.99181,35.74,140.00,千葉県,4.00,958.4000,76
1,10000269,35.70910,139.95892,35.70,139.96,千葉県,1.32,1172.6680,50
2,1020000002,35.71960,140.11927,35.72,140.12,千葉県,5.75,5150.5454,147
3,1110000001,35.72058,140.02484,35.72,140.02,千葉県,9.20,8432.7546,87
4,1110000010,35.65682,140.09113,35.66,140.10,千葉県,20.00,18208.5904,132
...,...,...,...,...,...,...,...,...,...
98,6910000421,35.73048,139.95546,35.74,139.96,千葉県,18.70,17351.7300,52
99,6910000424,35.74582,139.92609,35.74,139.92,千葉県,14.30,11921.5224,28
100,6910000425,35.73509,139.94345,35.74,139.94,千葉県,14.30,12474.2618,40
101,6910000469,35.81466,139.88526,35.82,139.88,埼玉県,14.30,13329.5162,8


In [46]:
use_nums = list(set(result_rounded.num.tolist()))

In [49]:
lat.to_numpy().reshape(-1)[use_nums]

array([35.66, 35.68, 35.78, 35.7 , 35.72, 35.74, 35.82, 35.72, 35.7 ,
       35.72, 35.74, 35.76, 35.78, 35.72, 35.8 , 35.84, 35.74, 35.76,
       35.78, 35.82, 35.86, 35.84, 35.7 , 35.74, 35.8 , 35.72, 35.84,
       35.88, 35.74, 35.78, 35.8 , 35.88, 35.68, 35.74, 35.82, 35.88,
       35.68, 35.72, 35.76, 35.8 , 35.68, 35.72, 35.78, 35.66, 35.82,
       35.7 , 35.72])

# 3_mesh_place

In [5]:
import os
import re

def get_file_paths(root_folder):
    file_paths = []
    for dirpath, dirnames, filenames in os.walk(root_folder):
        for filename in filenames:
            filepath = os.path.join(dirpath, filename)
            file_paths.append(filepath)
    return file_paths

root_folder = r"H:\study\preprocessing_data\3_mesh_place"

file_paths = get_file_paths(root_folder)
file_paths = [f for f in file_paths if len(f.split("\\"))==7]

In [11]:
import datetime 

df_list = []

for i,file_path in enumerate(file_paths):
    df = pd.read_csv(file_path, header=None, index_col=None)
    df = df.dropna()
    df.columns = ["id_lat_mesh", "id_lng_mesh", "nv_mesh"]

    year = "20" + file_path.split("\\")[-1][2:4]

    match = re.match(r'(\d{1,2})月(\d{1,2})日', file_path.split("\\")[-2])
    month = match.group(1)
    day = match.group(2)

    str_first = file_path.split("\\")[-1].split("_")[0]
    time = str_first[len("NV"+year[2:]+month+day):]
    hour = float(time)//1
    minute = float(time)%1*60

    date = datetime.datetime(int(year), int(month), int(day), int(hour), int(minute))

    df["datetime"] = date
    
    df_list.append(df)

    if i%int(len(file_paths)*0.1)==0:
        print(f"{i/len(file_paths)*100}%_finish")

    #break


0.0%_finish
10.0%_finish
20.0%_finish
30.0%_finish
40.0%_finish
50.0%_finish
60.0%_finish
70.0%_finish
80.0%_finish
90.0%_finish


In [20]:
s = pd.concat(df_list)
s.sort_values(ascending=[True,True,True],by=["datetime","id_lat_mesh","id_lng_mesh"],inplace=True)
s.reset_index(drop=True,inplace=True)
s.to_csv(OUTPUT_DIR + "mesh_place.csv",index=False)

In [23]:
t = pd.read_csv(OUTPUT_DIR + "mesh_place.csv")
t

,id_lat_mesh,id_lng_mesh,nv_mesh,datetime
0,31.50,131.06,1.000,2013-08-15 05:00:00
1,31.52,130.52,0.648,2013-08-15 05:00:00
2,31.66,130.32,0.777,2013-08-15 05:00:00
3,32.00,130.98,1.000,2013-08-15 05:00:00
4,32.02,130.74,0.982,2013-08-15 05:00:00
...,...,...,...,...
19755486,38.68,141.20,0.845,2014-07-31 18:30:00
19755487,38.70,141.10,0.871,2014-07-31 18:30:00
19755488,38.82,141.56,0.871,2014-07-31 18:30:00
19755489,38.90,141.58,0.871,2014-07-31 18:30:00
